In [22]:
import numpy as np 
import tensorflow as tf

#load data set
x = np.loadtxt("train_x.csv", delimiter=",") # load from text 
y = np.loadtxt("train_y.csv", delimiter=",") 
#x = x.reshape(-1, 64, 64) # reshape 
y = y.reshape(-1, 1) 

NameError: name 'dfloat32' is not defined

In [23]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer

classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 24, 25, 27, 28, 30, 32, 35, 36, 40, 42, 45, 48, 49, 54, 56, 63, 64, 72, 81]

y=LabelBinarizer().fit_transform(y).astype(float)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

#normalize value in x_train
x_train = x_train/255
x_test = x_test/255


array([[ 0.7372549 ,  0.69019608,  0.59607843, ...,  0.4627451 ,
         0.73333333,  0.4627451 ],
       [ 0.16078431,  0.23137255,  0.1254902 , ...,  0.18039216,
         0.18431373,  0.17647059],
       [ 0.47058824,  0.54509804,  0.54509804, ...,  0.56862745,
         0.56862745,  0.55686275],
       ..., 
       [ 0.7372549 ,  0.67058824,  0.81176471, ...,  0.54509804,
         0.70980392,  0.70588235],
       [ 0.18823529,  0.14117647,  0.1254902 , ...,  0.21960784,
         0.14901961,  0.17254902],
       [ 0.16470588,  0.16078431,  0.21960784, ...,  0.11372549,
         0.11764706,  0.11764706]])

In [36]:
#Convolutional Neural Network using Tensorflow
#labels is [0,1,2,....81]
def cnn_model_fn(features, labels, mode):
    
    #Model function for CNN.
    # Input Layer
    # [batch_size, image_width, image_height, channels]
    input_layer = tf.reshape(features["x"], [-1, 64, 64, 1])

    # Convolutional Layer #1
    # Input Tensor Shape: [batch_size, 64, 64, 1]
    # Output Tensor Shape: [batch_size, 64, 64, 32]
    conv1 = tf.layers.conv2d(inputs=input_layer,
                             filters=32,
                             kernel_size=[5, 5],
                             padding="same",
                             activation=tf.nn.relu)

    # Pooling Layer #1
    # Input Tensor Shape: [batch_size, 64, 64, 32]
    # Output Tensor Shape: [batch_size, 32, 32, 32]
    pool1 = tf.layers.max_pooling2d(inputs=conv1,
                                    pool_size=[2, 2],
                                    strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    # Input Tensor Shape: [batch_size, 32, 32, 32]
    # Output Tensor Shape: [batch_size, 32, 32, 64]
    conv2 = tf.layers.conv2d(inputs=pool1,
                             filters=64,
                             kernel_size=[5, 5],
                             padding="same",
                             activation=tf.nn.relu)
    # Input Tensor Shape: [batch_size, 32, 32, 64]
    # Output Tensor Shape: [batch_size, 16, 16, 64]
    pool2 = tf.layers.max_pooling2d(inputs=conv2,
                                    pool_size=[2, 2],
                                    strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 16 * 16 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    # Dropout to prevent overfitting
    # Output Tensor Shape: [batch_size, 1024]
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=40)

    # Generate predictions (for PREDICT and EVAL mode)
    # Add `softmax_tensor` to the graph. It is used for PREDICT and by the `logging_hook`.
      
    predictions = {"classes": tf.argmax(input=logits, axis=1),
                   "probabilities": tf.nn.softmax(logits, name="softmax_tensor")}

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    #print(labels.shape)
    #onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=40)
    
    loss = tf.losses.softmax_cross_entropy(onehot_labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss,
                                      global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [37]:
#Create estimator
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,model_dir="/tmp/mnist_cnn_model")

#Set up Logging Hook
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

#Start training model
x_train = np.float32(x_train)
x_test = np.float32(x_test)
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":x_train},y=y_train,batch_size=100,num_epochs=None,shuffle=True)
mnist_classifier.train(input_fn=train_input_fn,steps=20000,hooks=[logging_hook])

#Start evaluating model 
eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x":x_test},y=y_test,batch_size=100,num_epochs=None,shuffle=True)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)
                                                   
                                                   
                                                   

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_cnn_model', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_cnn_model/model.ckpt.
INFO:tensorflow:probabilities = [[ 0.02362376  0.02438007  0.02570157 ...,  0.02380401  0.0260332
   0.02666386]
 [ 0.02399162  0.0215299   0.02924267 ...,  0.02023645  0.02442562
   0.02326245]
 [ 0.0243233   0.01893912  0.02384566 ...,  0.02598598  0.02630477
   0.02298602]
 ..., 
 [ 0.02244388  0.02221     0.02851062 ...,  0.02207788  0.02336048
   0.0242826 ]
 [ 0.02275886  0.02235403  0.02844518 ...,  0.01994957  0.01984247
   0.01967052]
 [ 0.0278725   0.02043213  0.02537604 ...,  0.0230935   0.02058437
   0.0214065 ]]
IN

KeyboardInterrupt: 